# **Extracción, Transformación y Carga (ETL)**

En este proceso de Extracción, Transformación y Carga (ETL), nos encontraremos con cuatro tablas. Dos de estas tablas contienen datos recolectados sobre Homicidios por accidente de tránsito, cada una compuesta por una tabla de hechos y una tabla de víctimas. Del mismo modo, trabajaremos con datos recolectados sobre Lesionados por accidentes de tránsito, también con una tabla de hechos y una de víctimas.

## **1. Entendimiento del problema**

- Investigamos para comprender el problema de la seguridad vial en CABA y la importancia de reducir los siniestros viales.
- Definimos objetivos para identificar patrones, tendencias y áreas de mejora en la seguridad vial.

## **2. Adquisición de los datos**

- Carga de datos.

## **3. Exploración inicial de las tablas**

- Visualización de una muestra de los datos para comprender su estructura.
- Identificamos problemas de calidad del dato como valores faltantes, atípicos, duplicados, etc.

## **4. Limpieza de datos**

- Abordaremos los problemas identificados durante la exploración inicial de los datos.
- Realizaremos transformaciones de datos según sea necesario para preparar el dataset para el análisis.


####  -- Estructuramos el proceso de analisis de los datos en el siguiente orden:
1. **Homicidios - Hechos** 
2. **Homicidios - Victimas**
3. **Lesiones - Hechos**
4. **Lesiones - Victimas**




---
### **# Importamos librerias**


In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np


In [2]:
from funciones import cargar_datos_desde_excel
from funciones import analizar_SD
from funciones import analizar_nan
from funciones import data_cleaning
from funciones import analizar_palabra_clave
from funciones import completar_coordenadas_con_comuna
from funciones import analisis_categorico




### **Cargamos los Datasets**

In [3]:
# Cargamos TABLAS DE Lesiones - Hechos y Victimas
datos_lesiones = cargar_datos_desde_excel('../Datasets/1_raw_data/lesiones.xlsx', ['HECHOS', 'VICTIMAS'])
lesiones_hechos_ = datos_lesiones['HECHOS']
lesiones_victimas_ = datos_lesiones['VICTIMAS']



---
## **Exploración inicial de las tablas**
### **TABLA Lesiones - Hechos**

#### vista rapida a los datos


In [4]:

lesiones_hechos_.head(3)

,id_hecho,nro_victimas,fecha,hora,tipo_calle,comuna,longitud,latitud,victima,acusado,rol
0,LC-2019-0000179,1,2019-01-01 00:00:00,9,desconocido,14,-58.408911,-34.559658,CICLISTA,SD,desconocido
1,LC-2019-0000053,1,2019-01-01 00:00:00,1,desconocido,8,-58.44351,-34.669125,AUTO,SD,desconocido
2,LC-2019-0000063,1,2019-01-01 00:00:00,2,desconocido,8,-58.468335,-34.677556,SD,SD,desconocido


In [5]:
lesiones_victimas_.head(3)

,id_hecho,sexo,edad,gravedad
0,HC-2020-0129167,masculino,29,leve
1,HC-2020-0499647,masculino,19,leve
2,HC-2020-0499647,femenino,44,leve


#### Descripción estadística de los datos


In [6]:
lesiones_hechos_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id_hecho      23785 non-null  object
 1   nro_victimas  23785 non-null  int64 
 2   fecha         23785 non-null  object
 3   hora          23785 non-null  int64 
 4   tipo_calle    23785 non-null  object
 5   comuna        23616 non-null  object
 6   longitud      23523 non-null  object
 7   latitud       23523 non-null  object
 8   victima       23785 non-null  object
 9   acusado       23785 non-null  object
 10  rol           23785 non-null  object
dtypes: int64(2), object(9)
memory usage: 2.0+ MB


In [7]:

lesiones_hechos_.describe()

,nro_victimas,hora
count,23785.000000,23785.000000
mean,1.160563,13.606895
std,0.551571,5.553922
min,0.000000,0.000000
25%,1.000000,10.000000
50%,1.000000,14.000000
75%,1.000000,18.000000
max,16.000000,23.000000


#### Creamos un DataFrame que cuenta los valores NaN y su porcentaje representativo respecto a la columna

In [8]:

analizar_nan(lesiones_hechos_)

,Columna,Cantidad de NaN,% NaN x Columna
5,comuna,169,0.71
6,longitud,262,1.10
7,latitud,262,1.10


#### Creamos un DataFrame que cuenta los valores SIN DATO y su porcentaje representativo respecto a la columna

In [9]:
analizar_SD(lesiones_hechos_)

,Columna,Cantidad de SD,% SD x Columna
5,comuna,846,3.56
6,longitud,1209,5.08
7,latitud,1209,5.08
8,victima,10733,45.13
9,acusado,15288,64.28


#### Contar los valores duplicados en todo el DataFrame


In [10]:
lesiones_hechos_.duplicated().sum()

0

#### Conteo de valores únicos en variables categóricas

In [11]:
# Conteo de Comunas
cat_comuna_h = lesiones_hechos_['comuna'].astype('category').value_counts().sort_index().reset_index()

# Renombrar columnas
cat_comuna_h.columns = ['Comuna', 'Incidentes']

# Mostrar DataFrame resultante
cat_comuna_h


,Comuna,Incidentes
0,1,2526
1,2,878
2,3,1718
3,4,1729
4,5,1226
5,6,957
6,7,1516
7,8,1062
8,9,1674
9,10,1347


In [12]:
# Conteo de Victimas 
cat_victimas_h= lesiones_hechos_['victima'].value_counts().sort_index().reset_index()
cat_victimas_h.columns = ['Tipo de Victima', 'Incidentes']

cat_victimas_h

,Tipo de Victima,Incidentes
0,AUTO,2033
1,CAMION,79
2,CAMIONETA,189
3,CICLISTA,2209
4,MIXTO,42
5,MONOPATIN,28
6,MOTO,5461
7,MOVIL,100
8,OTRO,13
9,PEATON,2026


In [13]:
# Conteo de Acusados  
cat_acusado_h= lesiones_hechos_['acusado'].value_counts().sort_index().reset_index()
cat_acusado_h.columns = ['Tipo de Acusado', 'Incidentes']

cat_acusado_h

,Tipo de Acusado,Incidentes
0,AUTO,4641
1,CAMION,288
2,CAMIONETA,872
3,CICLISTA,77
4,MONOPATIN,3
5,MOTO,625
6,MOVIL,81
7,OBJETO FIJO,279
8,OTRO,24
9,PEATON,13


---
# **Limpieza Y Transformacion de Datos**
### **Lesiones - Hechos**


#### Analizamos las columnas que tienen valores NaN

In [14]:
analizar_nan(lesiones_hechos_)

,Columna,Cantidad de NaN,% NaN x Columna
5,comuna,169,0.71
6,longitud,262,1.10
7,latitud,262,1.10


#### Cambiamos el formato de la columna fecha de 2019-01-01 00:00:00 a 2019-01-01 

In [15]:
lesiones_hechos_= lesiones_hechos_.replace(['sd','SD'], np.nan)

/var/folders/jx/mgrpvh4n2f3b1l736sh02rpw0000gn/T/ipykernel_23961/4211815306.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lesiones_hechos_= lesiones_hechos_.replace(['sd','SD'], np.nan)


In [16]:
lesiones_hechos_['fecha'] = pd.to_datetime(lesiones_hechos_['fecha']).dt.date
lesiones_hechos_['hora'] = lesiones_hechos_['hora'].astype('int')

#### Pasamos todos los valores irregulares a NaN

In [17]:
lesiones_hechos_ = lesiones_hechos_.replace(['SD','sd','No Especificada'], np.nan).infer_objects(copy=False)


/var/folders/jx/mgrpvh4n2f3b1l736sh02rpw0000gn/T/ipykernel_23961/4133896823.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lesiones_hechos_ = lesiones_hechos_.replace(['SD','sd','No Especificada'], np.nan).infer_objects(copy=False)


#### observamos las Comunas disponibles y que informacion irregular hay

In [18]:
# Conteo de Comunas
cat_comuna_h = lesiones_hechos_['comuna'].astype('category').value_counts().sort_index().reset_index()

# Renombrar columnas
cat_comuna_h.columns = ['Comuna', 'Incidentes']

# Mostrar DataFrame resultante
cat_comuna_h


,Comuna,Incidentes
0,1.0,2526
1,2.0,878
2,3.0,1718
3,4.0,1729
4,5.0,1226
5,6.0,957
6,7.0,1516
7,8.0,1062
8,9.0,1674
9,10.0,1347


#### llenamos los datos de las horas faltantes con el dato no nulo anterior 

In [19]:
lesiones_hechos_['hora'] = lesiones_hechos_['hora'].ffill()


#### reemplazamos los valores Nulos de la columna 'Comuna' con la comuna con mas accidentes

In [20]:

# Encuentra la comuna más frecuente
comuna_mas_frecuente = lesiones_hechos_['comuna'].mode()[0]

# Reemplaza los valores NaN en la columna 'comuna' con la comuna más frecuente
lesiones_hechos_['comuna'] = lesiones_hechos_['comuna'].fillna(comuna_mas_frecuente)


In [21]:
lesiones_hechos_

,id_hecho,nro_victimas,fecha,hora,tipo_calle,comuna,longitud,latitud,victima,acusado,rol
0,LC-2019-0000179,1,2019-01-01,9,desconocido,14.0,-58.408911,-34.559658,CICLISTA,NaN,desconocido
1,LC-2019-0000053,1,2019-01-01,1,desconocido,8.0,-58.443510,-34.669125,AUTO,NaN,desconocido
2,LC-2019-0000063,1,2019-01-01,2,desconocido,8.0,-58.468335,-34.677556,NaN,NaN,desconocido
3,LC-2019-0000079,1,2019-01-01,2,desconocido,7.0,-58.437425,-34.647349,PEATON,NaN,desconocido
4,LC-2019-0000082,4,2019-01-01,4,desconocido,3.0,-58.398225,-34.604579,AUTO,NaN,desconocido
...,...,...,...,...,...,...,...,...,...,...,...
23780,LC-2021-0652849,1,2021-12-31,19,avenida,9.0,-58.513477,-34.659714,NaN,NaN,desconocido
23781,LC-2021-0652865,2,2021-12-31,19,avenida,9.0,-58.488327,-34.641753,NaN,NaN,desconocido
23782,LC-2021-0652907,1,2021-12-31,20,desconocido,1.0,-58.382894,-34.583083,NaN,NaN,desconocido
23783,LC-2021-0652921,1,2021-12-31,22,calle,5.0,-58.414532,-34.614288,MOTO,TRANSPORTE PUBLICO,desconocido


### Vamos a utilizar las coordenadas de la comuna con mas accidente para llenar los valores NaN de las columnas 'longitud' y 'latitud'

In [22]:
# Encuentra la comuna más frecuente
comuna_mas_frecuente = lesiones_hechos_['comuna'].mode()[0]

# Filtra el DataFrame para obtener los valores de longitud y latitud correspondientes a la comuna más frecuente
longitud_comuna_frecuente = lesiones_hechos_.loc[lesiones_hechos_['comuna'] == comuna_mas_frecuente, 'longitud'].dropna().iloc[0]
latitud_comuna_frecuente = lesiones_hechos_.loc[lesiones_hechos_['comuna'] == comuna_mas_frecuente, 'latitud'].dropna().iloc[0]

# Rellena los valores NaN en las columnas 'longitud' y 'latitud' con los valores obtenidos
lesiones_hechos_['longitud'] = lesiones_hechos_['longitud'].fillna(longitud_comuna_frecuente)
lesiones_hechos_['latitud'] = lesiones_hechos_['latitud'].fillna(latitud_comuna_frecuente)

#### Normalizaciones

In [23]:
# Cambiamos el tipo de dato de comuna y victimas
lesiones_hechos_[['comuna','nro_victimas']] = lesiones_hechos_[['comuna', 'nro_victimas']].astype(int)


In [24]:
# Pasamos las columnas a minúsculas
Columnas_minusculas = ['victima', 'acusado']
lesiones_hechos_ = data_cleaning(lesiones_hechos_, lowercase_columns=Columnas_minusculas)

# Pasamos los valores NaN de las columnas 'victima' y 'acusado' a 'desconocido'
lesiones_hechos_['victima'] = lesiones_hechos_['victima'].fillna('desconocido')
lesiones_hechos_['acusado'] = lesiones_hechos_['acusado'].fillna('desconocido')

# Columnas a categorizar
colum_categorizadas = ['tipo_calle','comuna', 'victima', 'acusado', 'rol']

# Categorizamos las columnas
for columna in colum_categorizadas:
    if columna in lesiones_hechos_.columns:
        lesiones_hechos_[columna] = lesiones_hechos_[columna].astype('category')



In [25]:
lesiones_hechos_

,id_hecho,nro_victimas,fecha,hora,tipo_calle,comuna,longitud,latitud,victima,acusado,rol
0,LC-2019-0000179,1,2019-01-01,9,desconocido,14,-58.408911,-34.559658,ciclista,desconocido,desconocido
1,LC-2019-0000053,1,2019-01-01,1,desconocido,8,-58.443510,-34.669125,auto,desconocido,desconocido
2,LC-2019-0000063,1,2019-01-01,2,desconocido,8,-58.468335,-34.677556,desconocido,desconocido,desconocido
3,LC-2019-0000079,1,2019-01-01,2,desconocido,7,-58.437425,-34.647349,peaton,desconocido,desconocido
4,LC-2019-0000082,4,2019-01-01,4,desconocido,3,-58.398225,-34.604579,auto,desconocido,desconocido
...,...,...,...,...,...,...,...,...,...,...,...
23780,LC-2021-0652849,1,2021-12-31,19,avenida,9,-58.513477,-34.659714,desconocido,desconocido,desconocido
23781,LC-2021-0652865,2,2021-12-31,19,avenida,9,-58.488327,-34.641753,desconocido,desconocido,desconocido
23782,LC-2021-0652907,1,2021-12-31,20,desconocido,1,-58.382894,-34.583083,desconocido,desconocido,desconocido
23783,LC-2021-0652921,1,2021-12-31,22,calle,5,-58.414532,-34.614288,moto,transporte publico,desconocido


---
## **Exploración inicial de las tablas**
### **TABLA Lesiones - Victimas**

In [26]:
lesiones_victimas_.head(3)

,id_hecho,sexo,edad,gravedad
0,HC-2020-0129167,masculino,29,leve
1,HC-2020-0499647,masculino,19,leve
2,HC-2020-0499647,femenino,44,leve


In [27]:
lesiones_victimas_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id_hecho  27605 non-null  object
 1   sexo      27605 non-null  object
 2   edad      27605 non-null  object
 3   gravedad  27605 non-null  object
dtypes: object(4)
memory usage: 862.8+ KB


In [28]:
lesiones_victimas_.describe()

,id_hecho,sexo,edad,gravedad
count,27605,27605,27605,27605
unique,23785,4,187,2
top,LC-2019-0188476,masculino,sd,leve
freq,16,17022,4471,27068


#### Creamos un DataFrame que cuenta los valores NaN y su porcentaje representativo respecto a la columna

In [29]:

analizar_nan(lesiones_victimas_)

,Columna,Cantidad de NaN,% NaN x Columna


#### Creamos un DataFrame que cuenta los valores SIN DATO y su porcentaje representativo respecto a la columna

In [30]:
analizar_palabra_clave(lesiones_victimas_, 'sd')

,Columna,Cantidad de sd,% sd x Columna
1,sexo,2228,8.07
2,edad,4471,16.20


In [31]:
analizar_palabra_clave(lesiones_victimas_, 'SD')

,Columna,Cantidad de SD,% SD x Columna


In [32]:
analizar_nan(lesiones_victimas_)

,Columna,Cantidad de NaN,% NaN x Columna


#### Contar los valores duplicados en todo el DataFrame


In [33]:
lesiones_victimas_.duplicated().sum()

633

#### borramos valores duplicados

In [34]:
lesiones_victimas_ = lesiones_victimas_.drop_duplicates()


#### Conteo de valores únicos en variables categóricas ACA


In [35]:
# Conteo de Sexo
cat_sexo= lesiones_victimas_['sexo'].value_counts().sort_index().reset_index()

cat_sexo.columns = ['Sexo', 'Frecuencia']
cat_sexo

,Sexo,Frecuencia
0,femenino,8168
1,femenino,1
2,masculino,16695
3,sd,2108


---
# **Limpieza Y Transformacion de Datos**
### **Lesiones - Victimas**


In [36]:
lesiones_victimas_

,id_hecho,sexo,edad,gravedad
0,HC-2020-0129167,masculino,29,leve
1,HC-2020-0499647,masculino,19,leve
2,HC-2020-0499647,femenino,44,leve
3,HC-2020-0499647,masculino,60,leve
4,HC-2020-0624349,femenino,41,leve
...,...,...,...,...
27600,PFA-2019-0000301,masculino,34,leve
27601,PFA-2019-0000314,masculino,27,leve
27602,PFA-2019-0001382,masculino,40,leve
27603,PFA-2019-0002588,masculino,40,leve


#### tenemos en la categoria 'Sexo' 2 categorias duplicadas para Femenino, asi que las mergeremos juntas.

In [37]:
# Hacer una copia del DataFrame para evitar SettingWithCopyWarning
lesiones_victimas_ = lesiones_victimas_.copy()

# Convertir todos los valores de 'sexo' a minúsculas
lesiones_victimas_['sexo'] = lesiones_victimas_['sexo'].str.lower()

# Reemplazar los valores diferentes por una única forma normalizada
lesiones_victimas_['sexo'] = lesiones_victimas_['sexo'].replace({'femenino ': 'femenino'})

# Verificar los valores únicos después de la limpieza
valores_unicos_despues = lesiones_victimas_['sexo'].unique()
print(valores_unicos_despues)


['masculino' 'femenino' 'sd']


#### vamos a reemplazar con la media los valores 'Sin Dato' en la columna 'Edad'

In [38]:
'''
# Calcula la media de las edades excluyendo los valores 'sd'
media_edades = lesiones_victimas_[lesiones_victimas_['edad'] != 'sd']['edad'].astype(float).mean()

# Redondea la media al entero más cercano
media_edades = int(round(media_edades))

# Reemplaza los valores 'sd' en la columna de 'edad' con la media calculada
lesiones_victimas_['edad'] = lesiones_victimas_['edad'].replace('sd', media_edades)

# Convierte la columna de 'edad' de vuelta a tipo entero
lesiones_victimas_['edad'] = pd.to_numeric(lesiones_victimas_['edad'], downcast='integer')
'''

"\n# Calcula la media de las edades excluyendo los valores 'sd'\nmedia_edades = lesiones_victimas_[lesiones_victimas_['edad'] != 'sd']['edad'].astype(float).mean()\n\n# Redondea la media al entero más cercano\nmedia_edades = int(round(media_edades))\n\n# Reemplaza los valores 'sd' en la columna de 'edad' con la media calculada\nlesiones_victimas_['edad'] = lesiones_victimas_['edad'].replace('sd', media_edades)\n\n# Convierte la columna de 'edad' de vuelta a tipo entero\nlesiones_victimas_['edad'] = pd.to_numeric(lesiones_victimas_['edad'], downcast='integer')\n"

In [39]:
lesiones_victimas_['edad'] = lesiones_victimas_['edad'].replace('sd', np.nan)



In [40]:
analizar_nan(lesiones_victimas_)

,Columna,Cantidad de NaN,% NaN x Columna
2,edad,4206,15.59


#### Reemplazar los 'sd' con 'desconocido' en la columna 'sexo'


In [41]:
# Reemplazar los 'sd' con 'desconocido' en la columna 'sexo'
lesiones_victimas_['sexo'] = lesiones_victimas_['sexo'].replace('sd', 'desconocido')

#### Categorizamos algunas columnas

In [42]:
# Columnas que queremos categorizar
colum_categorizadas = ['id_hecho', 'sexo', 'gravedad']

# Categorizamos las columnas
for columna in colum_categorizadas:
    if columna in lesiones_victimas_.columns:
        lesiones_victimas_[columna] = lesiones_victimas_[columna].astype('category')
    else:
        print(f"La columna '{columna}' no existe en el DataFrame.")

# Combinamos los datos

In [43]:
lesiones = pd.merge(lesiones_hechos_, lesiones_victimas_, left_on='id_hecho', right_on='id_hecho', how='inner')



In [44]:
# Reemplazar 'desconocido' con NaN en la columna 'sexo'
lesiones = lesiones.replace('desconocido', np.nan)

/var/folders/jx/mgrpvh4n2f3b1l736sh02rpw0000gn/T/ipykernel_23961/3858042995.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  lesiones = lesiones.replace('desconocido', np.nan)


In [45]:
analizar_nan(lesiones)

,Columna,Cantidad de NaN,% NaN x Columna
4,tipo_calle,12401,45.99
8,victima,12397,45.97
9,acusado,17376,64.44
10,rol,26965,100.00
11,sexo,2108,7.82
12,edad,4206,15.60


In [46]:
lesiones

,id_hecho,nro_victimas,fecha,hora,tipo_calle,comuna,longitud,latitud,victima,acusado,rol,sexo,edad,gravedad
0,LC-2019-0000179,1,2019-01-01,9,NaN,14,-58.408911,-34.559658,ciclista,NaN,NaN,NaN,NaN,leve
1,LC-2019-0000053,1,2019-01-01,1,NaN,8,-58.443510,-34.669125,auto,NaN,NaN,masculino,57,leve
2,LC-2019-0000063,1,2019-01-01,2,NaN,8,-58.468335,-34.677556,NaN,NaN,NaN,NaN,NaN,leve
3,LC-2019-0000079,1,2019-01-01,2,NaN,7,-58.437425,-34.647349,peaton,NaN,NaN,masculino,NaN,leve
4,LC-2019-0000082,4,2019-01-01,4,NaN,3,-58.398225,-34.604579,auto,NaN,NaN,masculino,45,leve
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26960,LC-2021-0652865,2,2021-12-31,19,avenida,9,-58.488327,-34.641753,NaN,NaN,NaN,masculino,28,leve
26961,LC-2021-0652865,2,2021-12-31,19,avenida,9,-58.488327,-34.641753,NaN,NaN,NaN,masculino,52,leve
26962,LC-2021-0652907,1,2021-12-31,20,NaN,1,-58.382894,-34.583083,NaN,NaN,NaN,masculino,23,leve
26963,LC-2021-0652921,1,2021-12-31,22,calle,5,-58.414532,-34.614288,moto,transporte publico,NaN,masculino,31,grave


In [47]:
lesiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26965 entries, 0 to 26964
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   id_hecho      26965 non-null  object  
 1   nro_victimas  26965 non-null  int64   
 2   fecha         26965 non-null  object  
 3   hora          26965 non-null  int64   
 4   tipo_calle    14564 non-null  category
 5   comuna        26965 non-null  category
 6   longitud      26965 non-null  float64 
 7   latitud       26965 non-null  float64 
 8   victima       14568 non-null  category
 9   acusado       9589 non-null   category
 10  rol           0 non-null      category
 11  sexo          24857 non-null  category
 12  edad          22759 non-null  object  
 13  gravedad      26965 non-null  category
dtypes: category(7), float64(2), int64(2), object(3)
memory usage: 1.6+ MB


### Guardamos la tabla limpia

In [48]:
# Almacenar los DataFrames finales en un nuevo archivo CSV

lesiones.to_csv('../Datasets/2_cleaned_data/lesiones_limpios.csv', index=False)

